# Imports

In [1]:
from utils.gui import interactive_segmentation_with_config

In [ ]:
interactive_segmentation_with_config()

Selected processing mode: ramified
Loaded stack with shape (192, 1536, 1536)


Running initial ramified segmentation...
Initial memory usage: 3134.22 MB

Starting first pass microglia segmentation with anisotropy normalization degree = 1...
Spacing (z, y, x): [1, 0.30690104166666665, 0.30690104166666665]
Target spacing: 1
Full zoom factors (z, y, x): [1.0, 3.258379295714892, 3.258379295714892]
Interpolated zoom factors (degree=1, z, y, x): [1.0, 0.30690104166666665, 0.30690104166666665]
Expected downsampled shape: (192, 471, 471)
New spacing (adjusted for degree): (1, 1.0, 1.0)
Downsampled volume shape: (192, 471, 471)
Smoothing volume...
Enhancing tubular structures with ultra-low memory usage...
Processing 192 slices with 1 scales
Processing scale [2.0, 2.0, 2.0] (1/1)...


Scale [2.0, 2.0, 2.0]: 100%|██████████| 192/192 [01:16<00:00,  2.53it/s]


Applying otsu thresholding with sensitivity 0.9...


100%|██████████| 4/4 [00:00<00:00, 191.95it/s]


...

Second-pass relabeling: 100%|██████████| 4/4 [00:01<00:00,  2.62it/s]


Relabeling for consecutive indices...


Applying new labels: 100%|██████████| 4/4 [00:01<00:00,  2.40it/s]


Found 122 connected components after stitching
Upsampling segmentation to original size...
Final memory usage: 5345.25 MB
Running initial ramified segmentation...
Initial memory usage: 5335.39 MB

Starting first pass microglia segmentation with anisotropy normalization degree = 1...
Spacing (z, y, x): [1, 0.30690104166666665, 0.30690104166666665]
Target spacing: 1
Full zoom factors (z, y, x): [1.0, 3.258379295714892, 3.258379295714892]
Interpolated zoom factors (degree=1, z, y, x): [1.0, 0.30690104166666665, 0.30690104166666665]
Expected downsampled shape: (192, 471, 471)
New spacing (adjusted for degree): (1, 1.0, 1.0)
Downsampled volume shape: (192, 471, 471)
Smoothing volume...
Enhancing tubular structures with ultra-low memory usage...
Processing 192 slices with 1 scales
Processing scale [2.0, 2.0, 2.0] (1/1)...


Scale [2.0, 2.0, 2.0]:  41%|████      | 79/192 [00:29<00:47,  2.40it/s]

In [ ]:
import os
import pandas as pd

# def organize_processing_dir(dir, mode):
"""
A function to organize the directory with multiple samples for processing with 3D segmentation
Inputs:
- dir: the directory containing tif files and a single csv file with the mapping of x,y,z dimensions to the corresponding tif files; csv file should have columns: 
Filename, Width (um), Height (um), Slices, Depth (um)

-mode: 'nuclear' or 'ramified', determines which config template to use
"""
#Make a list of all the tif files in the directory
tif_files = [f for f in os.listdir(dir) if f.endswith('.tif')]
#Make a list of all the csv files in the directory
csv_files = [f for f in os.listdir(dir) if f.endswith('.csv')]
#Check that there is only one csv file in the directory
if len(csv_files) != 1:
    raise ValueError('There should be only one csv file in the directory')
#Load the csv file
csv_file = csv_files[0]
csv_path = os.path.join(dir, csv_file)
df = pd.read_csv(csv_path)
#Check that the csv file has the correct columns
if not all([col in df.columns for col in ['Filename', 'Width (um)', 'Height (um)', 'Slices', 'Depth (um)']]):
    raise ValueError('The csv file should have columns: Filename, Width (um), Height (um), Slices, Depth (um)')
#Check that the csv file has the correct number of rows
if len(df) != len(tif_files):
    raise ValueError('The csv file should have the same number of rows as the number of tif files in the directory')
#Create a new directory for each tif file
root_names = [f.split('.')[0] for f in tif_files]
for root_name in root_names:
    new_dir = os.path.join(dir, root_name)
    os.makedirs(new_dir, exist_ok=True)
    #Move the tif file to the new directory
    tif_file = root_name + '.tif'
    tif_path = os.path.join(dir, tif_file)
    new_tif_path = os.path.join(new_dir, tif_file)
    os.rename(tif_path, new_tif_path)
    
    #Copy the correct yaml config file to the new directory
    if mode == 'nuclear':
        config_file = 'nuclear_config.yaml'
    elif mode == 'ramified':
        config_file = 'ramified_config.yaml'
    #Template config is in the same directory as this script
    config_path = os.path.join(os.path.dirname(os.path.realpath(__file__)), config_file)
    new_config_path = os.path.join(new_dir, config_file)
    os.system('cp {} {}'.format(config_path, new_config_path))
    
